# 概要
Demusar Significant Diabramの実行手順の説明

# で，Demusar Significant Diagramって何？
例えば，複数の機械学習モデルの性能を複数のデータを使って比較したいとき，データとの相性等の問題でモデルの性能が安定しない場合がある．

そのような場合にデータに依存せずモデルの性能を順位ベースで比較し，有意差検定を実施するための手法．

# で，何がうれしいの？
有意差の有無がグラフから見えるようになるので分かりやすい．

あと順位を使うので奥様うっとりノンパラメトリック検定．



### 以降，以下のデータ（実験結果）を用いて説明していく
比較したいモデルが5種類（modelA～E）あり，性能評価に用いるデータが6種類（dataA～F）ある場合を考える．

ここで**数値が小さいほうが性能が高い**指標を用いているとする．

In [ ]:
import pandas as pd
df = pd.DataFrame({'data_A' : [0.621, 0.661, 0.608, 0.821, 1.028],
                   'data_B' : [0.892, 0.820, 0.626, 0.828, 0.898],
                   'data_C' : [0.740, 0.911, 0.524, 0.821, 0.846],
                   'data_D' : [0.701, 0.628, 0.648, 0.764, 0.680],
                   'data_E' : [0.834, 1.042, 0.544, 0.810, 0.846],
                   'data_F' : [1.002, 0.862, 0.806, 0.884, 0.901]},
                   index=['Model_A','Model_B','Model_C','Model_D','Model_E'])
print(df)
# 行列が入れ替わっててちょっとわかりにくい．

表から，例えばmodel_Aでdata_Aを予測した場合の性能が0.621ということになる．

まず数値を列（データ）ごとの順位に変換し，行（モデル）ごとの平均値を算出する．変換した結果が以下．

なお，ここでは数値が小さければ順位が高い場合（昇順）を想定している．

逆に降順で順位を付ける場合には`model_rank=df.rank(ascending=False).mean(axis='columns')`とすること

In [ ]:
print('データごとのモデルの順位:')
print(df.rank())
model_rank=df.rank().mean(axis='columns')
print('モデルの平均順位:')
print(model_rank)

Nemenyi testで用いるCritical values算出のためのq_aの表を読み込む．表の出典は

https://kourentzes.com/forecasting/2014/05/01/critical-values-for-the-nemenyi-test/
    

In [ ]:
q_a_tb = pd.read_csv('http://kourentzes.com/forecasting/wp-content/uploads/2014/05/nemenyi.csv',index_col=0)

今回の実験に対応する$q_a$を取得する．今回のモデルは5種類で，有意水準は5%とする．

In [ ]:
q_a = q_a_tb.at[q_a_tb.index[len(df)-2],'Nemenyi 0.05']
print('q_a =', q_a)

CD (Critical Distance) を算出する．算出式は以下
$$
CD=q_a \times \sqrt{\frac{num\_model (num\_model+1)}{6 \times num\_data}}
$$
num_data: データの種類数，num_model: モデル数

式から分かるように，データの数が増えれば線は短く，モデルの数が増えれば線は長くなる．

In [ ]:
import math
CD = q_a * math.sqrt(len(df) * (len(df) + 1) / ( 6 * len(df.columns)))
print('CD =',CD)

グラフの描画

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
model_rs = model_rank.sort_values()
plt.scatter(model_rs,model_rs.index,c='black')
#点から伸びる直線（有意判定基準）の描画
for i, model in enumerate(model_rs):
    plt.plot([model,model+CD],[i,i], 'k-')
plt.axvline(model_rs[0]+CD, ls = "--", color = "black")

今回の場合，Model_Cのから上下に伸びる線の右端がModel_Eの左端より左にあるので，Model_CとModel_Eの間には有意差がある，ということになる．

# おわりに
自己責任でお使いください！